## [seq] import 

In [1]:
import json
import yaml
import sys
import time
import IPython.display as ipd
import pprint
from pathlib import Path
from tqdm import tqdm

import numpy as np
import torch
import torchaudio
from librosa.filters import mel as librosa_mel_fn
#import matplotlib
#matplotlib.use("Agg")
import matplotlib.pyplot as plt
from scipy.io.wavfile import write


import toybox

## [seq] check the configuration

## [seq] device setting

In [2]:
import os

print(f"all cpu at using device: {os.cpu_count()}")
print(f"Number of available CPU: {len(os.sched_getaffinity(0))}") # Number of available CPUs can also be obtained. ,use systemcall at linux.

all cpu at using device: 52
Number of available CPU: 4


In [3]:
DEVICE = 'cpu' # 'cuda' or 'cpu'
DEVICE_UTMOS = 'cuda'
if torch.cuda.is_available():
    print('use cuda')
else:
    os._exit(os.EX_OK)
    print('use cpu')

device = torch.device(DEVICE_UTMOS)
print(f'device: {device}')

use cuda
device: cuda


## [seq] load utmos model

In [4]:
# seed val from https://github.com/sarulab-speech/UTMOS22/blob/master/strong/configs/train/default.yaml
utmos_seed = 1234
toybox.set_seed(utmos_seed)

In [5]:
predictor_utmos = torch.hub.load("tarepan/SpeechMOS:v1.2.0", "utmos22_strong", trust_repo=True)

Using cache found in /work/sora-sa/.cache/torch/hub/tarepan_SpeechMOS_v1.2.0
/work/sora-sa/aoi/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


## [seq] setting path¶

In [6]:
test_ds_path = Path('configs/test_dataset.json')

LJWAV_DIR_PATH = Path('./data/ljspeech/LJSpeech-1.1/wavs/')
RESULT_DIR_PATH = Path('./result4eval/ljwav')
RESULT_JSON_PATH = RESULT_DIR_PATH / 'evallj1031.json'

In [7]:
print('-------------------------------------------')
if test_ds_path.exists():
    print(f'Exists {str(test_ds_path)}')
    with open(test_ds_path) as j:
        test_ds_list = json.load(j)
    print(f'loaded {test_ds_path}')
else:
    print(f'No exist {test_ds_path}')

print('-------------------------------------------')
if LJWAV_DIR_PATH.exists():
    print(f'Exists {LJWAV_DIR_PATH}')
else:
    print(f'No exist {LJWAV_DIR_PATH}')

print('-------------------------------------------')
if RESULT_DIR_PATH.exists():
    print(f'Exists {RESULT_DIR_PATH}')
else:
    RESULT_DIR_PATH.mkdir(parents=True)
    #print(f'No exist {RESULT_DIR_PATH}')

print('-------------------------------------------')
if RESULT_JSON_PATH.exists():
    print(f'Exists {RESULT_JSON_PATH}')
else:
    #RESULT_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_JSON_PATH}')

-------------------------------------------
Exists configs/test_dataset.json
loaded configs/test_dataset.json
-------------------------------------------
Exists data/ljspeech/LJSpeech-1.1/wavs
-------------------------------------------
-------------------------------------------
No exist result4eval/ljwav/evallj1031.json


## [seq] test load wav

In [8]:
ljwav_path = LJWAV_DIR_PATH / f"{test_ds_list[0]['name']}.wav"
#iwav_path = IWAV_DIR_PATH / f"{test_ds_list[0]['name']}.wav"
print(ljwav_path)
ljwav, ljsamplerate = torchaudio.load(ljwav_path)
ipd.display(ipd.Audio(ljwav, rate=ljsamplerate))

data/ljspeech/LJSpeech-1.1/wavs/LJ045-0049.wav


## [seq] test eval

In [9]:
# wav is [1, time]
score = predictor_utmos(ljwav, ljsamplerate)

In [10]:
print(score)
print(type(score))
print(score.item())
print(type(score.item()))

tensor([4.4567], grad_fn=<AddBackward0>)
<class 'torch.Tensor'>
4.45673942565918
<class 'float'>


## [seq] save LJ eval_json

In [11]:
infer_data_num: int = 100 #len(test_ds_list) is 200
print(infer_data_num)

100


In [12]:
print(RESULT_JSON_PATH)

result4eval/ljwav/evallj1031.json


In [13]:
score4lj_utmos_list = []
for i in tqdm(range(infer_data_num)):
    ljfilename = test_ds_list[i]['name']
    ljwav_path = Path('./data/ljspeech/LJSpeech-1.1/wavs') / f"{ljfilename}.wav"
    print(ljwav_path)
    ljwav, ljsamplerate = torchaudio.load(ljwav_path)
    score4lj_utmos = predictor_utmos(ljwav, ljsamplerate)
    score_float = score4lj_utmos.item()
    evallj_dict = {'name': ljfilename, 'path': str(ljwav_path), 'utmos': score_float}
    score4lj_utmos_list.append(evallj_dict)
        

  0%|                                                                                           | 0/100 [00:00<?, ?it/s]

data/ljspeech/LJSpeech-1.1/wavs/LJ045-0049.wav


  2%|█▋                                                                                 | 2/100 [00:00<00:28,  3.48it/s]

data/ljspeech/LJSpeech-1.1/wavs/LJ017-0027.wav
data/ljspeech/LJSpeech-1.1/wavs/LJ023-0031.wav


  3%|██▍                                                                                | 3/100 [00:00<00:21,  4.47it/s]

...

 98%|████████████████████████████████████████████████████████████████████████████████▎ | 98/100 [00:34<00:00,  2.52it/s]

data/ljspeech/LJSpeech-1.1/wavs/LJ018-0059.wav


 99%|█████████████████████████████████████████████████████████████████████████████████▏| 99/100 [00:34<00:00,  2.39it/s]

data/ljspeech/LJSpeech-1.1/wavs/LJ029-0081.wav


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:35<00:00,  2.84it/s]


In [14]:
if RESULT_JSON_PATH.exists() == False:
    with open(RESULT_JSON_PATH, 'w') as f:
        for entry in score4lj_utmos_list:
            f.write(json.dumps(entry) + '\n')
    print(f'Make {RESULT_JSON_PATH}')
else:
    print(f'Already Exists {RESULT_JSON_PATH}')

Make result4eval/ljwav/evallj1031.json


## [seq] eval wav

In [23]:
eval_target = 'lj_via_hifigan'

eval_info = {
    'lj_1013': './result4eval/ljwav/evallj1031.json',
    'lj_pbl': 'result4eval/infer4PBL/groundtruth/evalljPBL.json',
    'lj_via_hifigan': 'result4eval/infer4PBL/groundtruth/eval4PBL.json'
}

eval_jsonl_path = Path(eval_info[eval_target])
if eval_jsonl_path.exists() == True:
    print(f'Exist {eval_jsonl_path}')
    import json
    with open(eval_jsonl_path) as f:
        eval_jsonl_list = [json.loads(l) for l in f]
else:
    print(f'No Exists {eval_jsonl_path}')

Exist result4eval/infer4PBL/groundtruth/eval4PBL.json


In [24]:
utmos_list = [eval_jsonl_list[n]['utmos'] for n in range(len(eval_jsonl_list))]
utmos_nparr = np.array(utmos_list)

In [25]:
utmos_mean = np.mean(utmos_nparr)
utmos_var = np.var(utmos_nparr)
utmos_std = np.std(utmos_nparr)
print(f'utmos mean: {utmos_mean}')
print(f'utmos var: {utmos_var}')
print(f'utmos std: {utmos_std}')

utmos mean: 4.019014093875885
utmos var: 0.09124055665879827
utmos std: 0.30206051820586927
